# This notebook is now replaced by the script named runall.py. However, this notebook will serve as reference code to implement the inference tests. The inference tests will be performed on the model checkpoints to analyse network performance, network training progress and image quality.

# Code starts here

In [1]:
%matplotlib inline


In [2]:
# ACHTUNG: http://thomas-cokelaer.info/blog/2011/09/382/
# Errors when reloading module with the class in jupyter notebook !
from models import modelRepository as mr
from myCode import myFunctions
from myCode import myDataLoader
from models import UNet3d_parts
from models import UNet3d_assembled



In [3]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms


import numpy as np
import matplotlib.pyplot as plt
#import scipy
import scipy.io as spio

import os
import time


# Setup Tensorboard
### Tensorboard from TF 1.0 has been used, as TF 2.0 is not fully compatible with conda yet. This will avoid possible issues when copying the conda environment over to the computer cluster O2

In [4]:
# import tensorflow as tf
# from tensorflow import summary
from tensorboard import notebook
from tensorboardX import SummaryWriter
from tensorboardX import FileWriter

%load_ext tensorboard.notebook

In [ ]:
# import skorch
from skorch import NeuralNet
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error
from skorch.callbacks import EpochScoring
from sklearn.model_selection import train_test_split
import skorch.callbacks

import torchbearer


# Create the Dataset Class according to the following instructions from the pytorch "DATA LOADING AND PROCESSING TUTORIAL" notebook

In [5]:
class CMRIreconDataset(Dataset):
    """CMRIrecon dataset."""

    def __init__(self, input_file_path, target_file_path):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.inputs = np.load(input_file_path)
        self.targets = np.load(target_file_path)
    
    def __len__(self):
#         print("print length of inputs",len(self.inputs))
#         print("print shape of inputs",np.shape(self.inputs))
        return len(self.inputs)

    def __getitem__(self, idx):

#         sample = {'input': self.inputs[idx], 'target': self.targets[idx]}
        X = self.inputs[idx]
        Y = self.targets[idx]
        return  X, Y

In [6]:
CMRIdataset = CMRIreconDataset(
        input_file_path = \
        'C:/Users/littl/Documents/PythonScripts/reconproject_data/input_data.npy', \
        target_file_path = \
        'C:/Users/littl/Documents/PythonScripts/reconproject_data/target_data.npy')

# print(CMRIdataset[:]['input'].shape)
X, Y = CMRIdataset[:][:]

print(X.shape)
print(Y.shape)

(671, 20, 96, 96)
(671, 20, 96, 96)


## Split Dataset into train set ( 80% ) and validation set ( 20% )

## This Method is possibly not ideal yet, as the two generated dataset do not have the same amount of of slices from each heart layer.

In [7]:
train_size = int(0.8 * len(CMRIdataset))
val_size = len(CMRIdataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(CMRIdataset, [train_size, val_size])


# define loaders
# ! numworkers set to 0 for windows !!
# load training set
trainloader = DataLoader(train_dataset, batch_size=4,
                        shuffle=True, num_workers=0)

# load validation set
valloader = DataLoader(val_dataset, batch_size=4,
                        shuffle=True, num_workers=0)

# print(train_dataset[:]['input'].shape)
# print(val_dataset[:]['input'].shape)


# Note about NN input.
## c.f. NN tutorial from pytorch

``torch.nn`` only supports mini-batches. The entire ``torch.nn`` package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, ``nn.Conv2d`` will take in a 4D Tensor of
``nSamples x nChannels x Height x Width``.

If you have a single sample, just use ``input.unsqueeze(0)`` to add
a fake batch dimension.</p></div>

# Define ConvNet

## Import model from modelRepository.py

In [8]:
net = mr.BNv0()
net2 = mr.BNv1()


In [9]:
channels = 20
model = UNet3d_assembled.UNet3d(channels)
# override
model = net2


model = mr.BN20channels()
print(type(model))

<class 'models.modelRepository.BN20channels'>


In [10]:
# import torch
# torch.rand(1).cuda()
# # DOES NOT WORK

# This project is not yet compatible with train and val data at the same time 
# and can only overfit one dataset at a time.

In [12]:
# INCLUDE code for multiple GPUs !! #

# move computation to gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


# Define Data

In [13]:

# X, Y = CMRIdataset
X, Y = CMRIdataset[:5][:5]
# X = torch.from_numpy(X.astype(np.float32))
# Y = torch.from_numpy(Y.astype(np.float32))

X = X.astype(np.float32)
Y = Y.astype(np.float32)


print(type(model))
print(type(X))
print(type(Y))

print(X.shape)
print(Y.shape)

# X_train, X_val, Y_train, Y_val = train_test_split(X.numpy(), Y.numpy(), test_size = 0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2)

print(X_train.shape)

# X = torch.from_numpy(X).float.to(device)
# Y = torch.from_numpy(Y).float.to(device)

<class 'models.modelRepository.BN20channels'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(5, 20, 96, 96)
(5, 20, 96, 96)
(4, 20, 96, 96)


# Define Callback functions

In [14]:
# skorch_callbacks
# mean absolute error between train image and val image

# def mean_abs_error_train(x_train, y_train):
# #     NeuralNet.initialize_module(model)
#     y_pred = model.predict(x_train)
# #     Y_pred = model(x_train)
#     return mean_absolute_error(y_train, x_train)

print(X.shape)
print(Y.shape)
# print(model.shape)

def mean_abs_error_train(X, Y):
    Y_pred = model.predict(X)
    print(Y_pred.shape)
    return mean_absolute_error(Y, Y_pred)
#                         multioutput = 'raw_values'
                        

mean_abs_error_train_scorer = make_scorer(mean_abs_error_train)

epoch_MAE_train = EpochScoring(
        mean_abs_error_train_scorer,
        name = 'MAE_train',
        lower_is_better = True,
        on_train = True,
        use_caching = True
        )


# cbs = [('my score', epoch_MAE_train)]
cyclicLR = skorch.callbacks.LRScheduler(
            policy = 'CyclicLR',
            )
# lr = skorch.callbacks.CyclicLR(optimizer = Adam)
# progressbar = skorch.callbacks.ProgressBar()

(5, 20, 96, 96)
(5, 20, 96, 96)


In [15]:
model = mr.BN20channels()
print(type(model))

model = NeuralNet(module = model,
                criterion = nn.MSELoss,
                max_epochs = 5,
                lr = 0.003,
                device = 'cuda',
#                 callbacks = [('lr', lr)]
#                 callbacks = [('epochfoo', epoch_MAE_train)],
                callbacks = [
#                     ('progressbarfoo', progressbar), 
#                     ('epochfoo', epoch_MAE_train),
                        (cyclicLR),                    
                    ],

                )


<class 'models.modelRepository.BN20channels'>


In [16]:

model.initialize()

<class 'skorch.net.NeuralNet'>[initialized](
  module_=BN20channels(
    (conv1): Conv2d(20, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (BN32): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (BN64): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (deconv2): ConvTranspose2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (conv3): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (BN32_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(32, 20, kernel_size=(1, 1), stride=(1, 1))
    (BN20): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv5): Conv2d(20, 20, kernel_size=(1, 1), stride=(1, 1))
  ),
)

In [17]:
model.fit(X, Y)

Re-initializing optimizer because the following parameters were re-set: .
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        1.0736        1.0159  1.7596
      2        1.0728        1.0159  0.0409
      3        1.0721        1.0159  0.0399
      4        1.0714        1.0160  0.0423
      5        1.0706        1.0161  0.0449


<class 'skorch.net.NeuralNet'>[initialized](
  module_=BN20channels(
    (conv1): Conv2d(20, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (BN32): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (BN64): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (deconv2): ConvTranspose2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (conv3): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (BN32_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(32, 20, kernel_size=(1, 1), stride=(1, 1))
    (BN20): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv5): Conv2d(20, 20, kernel_size=(1, 1), stride=(1, 1))
  ),
)

In [30]:
# cyclicLR.get_scheduler(model)
cyclicLR.previous_epoch_train_loss_score(model)

AttributeError: 'LRScheduler' object has no attribute 'previous_epoch_train_loss_score'

In [18]:
model.set_params(max_epochs = 1000)

_ = model.partial_fit(X, Y)

      6        1.0699        1.0162  0.0419
      7        1.0692        1.0163  0.0409
      8        1.0685        1.0165  0.0429
      9        1.0678        1.0169  0.0459
     10        1.0671        1.0172  0.0429
     11        1.0664        1.0176  0.0429
     12        1.0657        1.0180  0.0379
     13        1.0650        1.0184  0.0519
     14        1.0643        1.0189  0.0379
     15        1.0637        1.0194  0.0369
     16        1.0630        1.0199  0.0718
     17        1.0624        1.0204  0.0369
     18        1.0617        1.0210  0.0364
     19        1.0611        1.0216  0.0359
     20        1.0605        1.0222  0.0359
     21        1.0598        1.0228  0.0359
     22        1.0592        1.0234  0.0359
     23        1.0586        1.0241  0.0359
     24        1.0580        1.0247  0.0353
     25        1.0575        1.0254  0.0349
     26        1.0569        1.0260  0.0359
     27        1.0563        1.0267  0.0349
     28        1.0558        1.0

    378        0.9414        0.9623  0.0350
    379        0.9411        0.9620  0.0338
    380        0.9408        0.9618  0.0330
    381        0.9404        0.9616  0.0350
    382        0.9401        0.9614  0.0349
    383        0.9398        0.9612  0.0340
    384        0.9395        0.9610  0.0349
    385        0.9392        0.9608  0.0340
    386        0.9388        0.9606  0.0350
    387        0.9385        0.9604  0.0359
    388        0.9382        0.9602  0.0349
    389        0.9379        0.9600  0.0359
    390        0.9375        0.9598  0.0349
    391        0.9372        0.9596  0.0349
    392        0.9369        0.9593  0.0389
    393        0.9366        0.9591  0.0359
    394        0.9362        0.9589  0.0339
    395        0.9359        0.9587  0.0350
    396        0.9356        0.9585  0.0339
    397        0.9352        0.9583  0.0339
    398        0.9349        0.9581  0.0339
    399        0.9346        0.9578  0.0349
    400        0.9343        0.9

    750        0.7382        0.8511  0.0340
    751        0.7374        0.8507  0.0340
    752        0.7365        0.8504  0.0334
    753        0.7356        0.8500  0.0340
    754        0.7348        0.8497  0.0332
    755        0.7339        0.8493  0.0340
    756        0.7330        0.8490  0.0342
    757        0.7321        0.8486  0.0361
    758        0.7312        0.8483  0.0350
    759        0.7304        0.8480  0.0353
    760        0.7295        0.8476  0.0340
    761        0.7286        0.8473  0.0354
    762        0.7277        0.8469  0.0339
    763        0.7268        0.8465  0.0339
    764        0.7259        0.8462  0.0349
    765        0.7250        0.8458  0.0349
    766        0.7241        0.8455  0.0340
    767        0.7232        0.8452  0.0340
    768        0.7223        0.8448  0.0340
    769        0.7214        0.8445  0.0340
    770        0.7205        0.8441  0.0349
    771        0.7196        0.8438  0.0349
    772        0.7187        0.8

In [20]:
# initialize_module(model)
model.module_(X)

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [21]:
Y_pred = model.predict(X)
print(Y_pred[1].shape)
print(Y_train.shape)
print(X_train.shape)

(20, 96, 96)
(4, 20, 96, 96)
(4, 20, 96, 96)


In [22]:
print(len(model.history))
print(model.history[-1, 'train_loss'])
print(model.history[-1, 'valid_loss'])

1005
0.5086947679519653
0.8109692335128784


In [ ]:
STOP CODE HERE



In [ ]:
alpha = 0.003

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=alpha)


#override previous
for batch in trainloader:
    input = batch['input']
    target = batch['target']

# move computation to gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# INCLUDE code for multiple GPUs !! #

model = model.double().to(device)
input = input.to(device)
target = target.to(device)

print("shape input", input.shape)
print("shape target", target.shape)

#tensorboardX setup
curr_time = time.strftime("%Y%m%d_%H%M%S", time.localtime())
logdir = os.path.join("logs", curr_time)
logdir_train = os.path.join(logdir, "train")
logdir_val = os.path.join(logdir, "val")
train_summary_writer = SummaryWriter(logdir_train) # train SummaryWriter
val_summary_writer = SummaryWriter(logdir_val)     # validation SummaryWriter

print("\n--------------------------------------------------")
print("current time marker: ", curr_time)
print("Tensorboard log directory location:", logdir)
print("--------------------------------------------------\n")


In [ ]:
#  THE TRAINER FUNCTION NEEDS TO BE ADAPTED FROM THE myTrainer.py FILE

In [ ]:
for epoch in range(100):  # loop over the dataset multiple times
    
    i = 0
    running_loss = 0.0
#     for i, data in enumerate(sample, 0):
        # get the inputs

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    output = model(input)
    
    loss = criterion(output, target)
    train_summary_writer.add_scalar('train_loss', loss.item(), epoch)
    loss.backward()
    optimizer.step()

    # print statistics
    print(epoch, "\t", loss.item())
#     running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0
#     print('[%d, %5d] loss: %.3f' %
#           (epoch + 1, i + 1, running_loss / 2000))
        
print('Finished Training')

In [ ]:
# output = best model (best val loss)
output = model(input)

In [ ]:
# Save files to matlab folder

In [ ]:
def savetomatlab(target_dir, datatype, datatypestr, sample_number, overwrite = False): # e.g. savetomatlab(foo_dir, input)
    
    if overwrite == False:
        print("Set overwrite = True to save the results !")
    
#   print avoids to mistakenly overwrite previous results.
    if overwrite == True:
        output_path = ('C:/Users/littl/Documents/PythonScripts/reconproject/cmriRecon/results/' + target_dir)

        if not os.path.exists(output_path):
            os.makedirs(output_path)

        spio.savemat(output_path + '/' + datatypestr + '_sample_' + str(sample_number), \
                {datatypestr + '_sample_' + str(sample_number) \
                 : datatype[sample_number,...]}) #dict w/ np value

        print("Result images saved at:\n", output_path )

In [ ]:
#dont forget to use:
# input = myFunctions.imprepare(input);
# output = myFunctions.imprepare(output);
# target = myFunctions.imprepare(target);


for i in range(len(input)):
    savetomatlab('foo5', input, 'input', i)
    savetomatlab('foo5', output, 'output', i)
    savetomatlab('foo5', target, 'target', i)

In [ ]:
print(input.shape)
print(output.shape)
print(target.shape)

# Plot training loss and validation loss

In [ ]:
# documentation at https://www.tensorflow.org/tensorboard/r2/tensorboard_in_notebooks
%tensorboard --logdir logs --port 6016

In [ ]:
# Control TensorBoard display. If no port is provided, the most recently launched TensorBoard is used
# notebook.display(port=6016)#, height=1000)
# notebook.list()

# Validate results

In [ ]:
# code for single sample loading

sample = CMRIdataset[100]
input = sample['input']
target = sample['target']

print("shape input", input.shape)
print("shape target", target.shape)

#to gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input = torch.from_numpy(input).float().unsqueeze(0).double().to(device)
target = torch.from_numpy(target).float().unsqueeze(0).double().to(device)
model = model.double().to(device)

# reconstruct validation image
output = model(input)
print(sample_output.size)

input = myFunctions.imprepare(input);
output = myFunctions.imprepare(output);
target = myFunctions.imprepare(target);

print(sample_output.size)

for i in range(len(input)):
    savetomatlab('foo4val', input, 'input', i)
    savetomatlab('foo4val', output, 'output', i)
    savetomatlab('foo4val', target, 'target', i)

# Display Results

In [ ]:
#DISPLAY IMAGES
foo = []
# reshape the images to fit them in the input
# still need to figure out how to produce list of images.
input_reshape = input.reshape([-1, 96, 96])
for i in range(len(input)):
    foo.append(input_reshape[i].cpu())
    
print(foo[0])

In [ ]:
print(input.shape)

# for i in range(len(input)):
#     input[:,i,:,:]
foo = []
input_reshape = input.reshape([-1, 96, 96]).unsqueeze(1)
print(input_reshape.shape)
for i in range(len(input)):
    foo.append(input_reshape[i].cpu())
    

grid_img = torchvision.utils.make_grid(input_reshape.cpu())

print(input.reshape([-1,96,96]).shape)
print(grid_img.shape)


plt.imshow(grid_img)

# plt.imshow(input.cpu().detach().numpy()[0,0,...])

# plt.imshow(output.cpu().detach().numpy()[0,0,...])
# plt.imshow(target.cpu().detach().numpy()[0,0,...])

In [ ]:
def imshow(npimg):
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

imshow(torchvision.utils.make_grid(torch.from_numpy(input)))
imshow(myFunctions.imprepare(torchvision.utils.make_grid(target)))
imshow(myFunctions.imprepare(torchvision.utils.make_grid(output)))